# Project 02 - No.8
## 網路股票討論度與波動度關係之探討

In [194]:
import requests
import time
from bs4 import BeautifulSoup
import os
import re
import urllib.request
import json
import jieba
import jieba.analyse
import pandas as pd
from pandas.core.frame import DataFrame

In [195]:
PTT_URL = 'https://www.ptt.cc'

In [196]:
def get_web_page(url):
    time.sleep(0.5)  # 每次爬取前暫停 0.5 秒以免被 PTT 網站判定為大量惡意爬取
    resp = requests.get(url=url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [197]:
def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').string.strip() == date:  # 發文日期正確

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').string
                articles.append(title+"~"+date)
    return articles, prev_url

In [198]:
def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    links = soup.find(id='main-content').find_all('a')
    img_urls = []
    for link in links:
        if re.match(r'^https?://(i.)?(m.)?imgur.com', link['href']):
            img_urls.append(link['href'])
    return img_urls


In [241]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Stock/index.html')
    if current_page:
        articles = []  # 全部的文章
        
    dates = ("01/06","01/05","01/04","01/03","01/02","01/01","12/31","12/30","12/29","12/28",
             "12/27","12/26","12/25",'12/24','12/23',"12/22","12/21","12/20","12/19","12/18",
             "12/17",'12/16','12/15','12/14','12/13',"12/12","12/11",'12/10',"12/09",'12/08','12/07','12/06')
    for c in dates:
        date = time.strftime(c).lstrip('0') # 去掉開頭的 '0' 以符合 PTT 網站日期格式
        current_articles, prev_url = get_articles(current_page, date)  # 目前頁面的文章
        while current_articles:  # 若目前頁面有文章則加入 articles，並回到上一頁繼續尋找是否有文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, date)
        for article in articles:
            print(article)
        

Re: [請益] 台積電上300買認購權證是不是可以大賺!?~1/06
[請益] 還有人在用權證想發財嗎~1/06
Re: [請益] 期貨程式交易 有人接觸嗎~1/06
[新聞] 陽明、長榮海迎接小旺季~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[標的] 3081聯亞~1/06
[新聞]登不進《天堂M》不爽了　課長爆氣刷退58萬~1/06
Re: [新聞]逢低布局蘋概股，GIS、美律權證亮~1/06
[公告] N5D6M6C 違反板規8 水桶10年~1/06
[公告] efun0242違反板規4-4 水桶一週~1/06
[請益] 近幾年像台積電一樣強勢的股票~1/06
[請益] 銀行保險業的動作，可判斷匯率走勢嗎？~1/06
[新聞] 友訊(2332)  利多帶動股價漲停~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 康控營收~1/06
[公告] bsexpb060137 違反板規4-4 水桶一週~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[請益] 愛榭克是不是台灣最強經濟分析師？~1/06
[新聞] 墾丁飯店歇業全怪蔡英文？財經專家曝倒~1/06
[心得] 百條心法系列：第一條~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 《天堂M》故障爆退款潮　課長們刷退破百~1/06
[請益] 財報狗網站~1/06
[請益] type-C在台灣真的有肉可以吃嗎?~1/06
[標的] 2395研華 多~1/06
[新聞] 《財經觀測站》跟著大股東進出準沒錯~1/06
[新聞] 進口車強勢進逼　裕隆執行長嚴凱泰：我~1/06
[標的] 6462 神盾~1/06
[新聞] 16檔大戶搶買 下周領攻11,000點~1/06
[新聞] 宏達電12月營收僅40億元，創近13年次低~1/06
Re: [請益] 台積電上300買認購權證是不是可以大賺!?~1/06
[請益] 還有人在用權證想發財嗎~1/06
Re: [請益] 期貨程式交易 有人接觸嗎~1/06
[新聞] 陽明、長榮海迎接小旺季~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[標的] 3081聯亞~1/06
[新聞]登不進《天堂M》

Re: [請益] 台積電上300買認購權證是不是可以大賺!?~1/06
[請益] 還有人在用權證想發財嗎~1/06
Re: [請益] 期貨程式交易 有人接觸嗎~1/06
[新聞] 陽明、長榮海迎接小旺季~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[標的] 3081聯亞~1/06
[新聞]登不進《天堂M》不爽了　課長爆氣刷退58萬~1/06
Re: [新聞]逢低布局蘋概股，GIS、美律權證亮~1/06
[公告] N5D6M6C 違反板規8 水桶10年~1/06
[公告] efun0242違反板規4-4 水桶一週~1/06
[請益] 近幾年像台積電一樣強勢的股票~1/06
[請益] 銀行保險業的動作，可判斷匯率走勢嗎？~1/06
[新聞] 友訊(2332)  利多帶動股價漲停~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 康控營收~1/06
[公告] bsexpb060137 違反板規4-4 水桶一週~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[請益] 愛榭克是不是台灣最強經濟分析師？~1/06
[新聞] 墾丁飯店歇業全怪蔡英文？財經專家曝倒~1/06
[心得] 百條心法系列：第一條~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 《天堂M》故障爆退款潮　課長們刷退破百~1/06
[請益] 財報狗網站~1/06
[請益] type-C在台灣真的有肉可以吃嗎?~1/06
[標的] 2395研華 多~1/06
[新聞] 《財經觀測站》跟著大股東進出準沒錯~1/06
[新聞] 進口車強勢進逼　裕隆執行長嚴凱泰：我~1/06
[標的] 6462 神盾~1/06
[新聞] 16檔大戶搶買 下周領攻11,000點~1/06
[新聞] 宏達電12月營收僅40億元，創近13年次低~1/06
[新聞] 國產車大降價 破格搶客 ~1/05
[新聞] 華孚2017Q4合併營收破9億 連6季刷新紀~1/05
Re: [新聞] 盤子笑了！花269萬終獲《天堂M》紫變~1/05
[心得] 現在只要喊漲價就可以飆！？~1/05
Re: [標的] 3025星通~1/05
Re: [心得] 現在只要喊漲價就可以飆！？~1/05
大盤解讀~1/0

Re: [請益] 台積電上300買認購權證是不是可以大賺!?~1/06
[請益] 還有人在用權證想發財嗎~1/06
Re: [請益] 期貨程式交易 有人接觸嗎~1/06
[新聞] 陽明、長榮海迎接小旺季~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[標的] 3081聯亞~1/06
[新聞]登不進《天堂M》不爽了　課長爆氣刷退58萬~1/06
Re: [新聞]逢低布局蘋概股，GIS、美律權證亮~1/06
[公告] N5D6M6C 違反板規8 水桶10年~1/06
[公告] efun0242違反板規4-4 水桶一週~1/06
[請益] 近幾年像台積電一樣強勢的股票~1/06
[請益] 銀行保險業的動作，可判斷匯率走勢嗎？~1/06
[新聞] 友訊(2332)  利多帶動股價漲停~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 康控營收~1/06
[公告] bsexpb060137 違反板規4-4 水桶一週~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[請益] 愛榭克是不是台灣最強經濟分析師？~1/06
[新聞] 墾丁飯店歇業全怪蔡英文？財經專家曝倒~1/06
[心得] 百條心法系列：第一條~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 《天堂M》故障爆退款潮　課長們刷退破百~1/06
[請益] 財報狗網站~1/06
[請益] type-C在台灣真的有肉可以吃嗎?~1/06
[標的] 2395研華 多~1/06
[新聞] 《財經觀測站》跟著大股東進出準沒錯~1/06
[新聞] 進口車強勢進逼　裕隆執行長嚴凱泰：我~1/06
[標的] 6462 神盾~1/06
[新聞] 16檔大戶搶買 下周領攻11,000點~1/06
[新聞] 宏達電12月營收僅40億元，創近13年次低~1/06
[新聞] 國產車大降價 破格搶客 ~1/05
[新聞] 華孚2017Q4合併營收破9億 連6季刷新紀~1/05
Re: [新聞] 盤子笑了！花269萬終獲《天堂M》紫變~1/05
[心得] 現在只要喊漲價就可以飆！？~1/05
Re: [標的] 3025星通~1/05
Re: [心得] 現在只要喊漲價就可以飆！？~1/05
大盤解讀~1/0

Re: [請益] 台積電上300買認購權證是不是可以大賺!?~1/06
[請益] 還有人在用權證想發財嗎~1/06
Re: [請益] 期貨程式交易 有人接觸嗎~1/06
[新聞] 陽明、長榮海迎接小旺季~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[標的] 3081聯亞~1/06
[新聞]登不進《天堂M》不爽了　課長爆氣刷退58萬~1/06
Re: [新聞]逢低布局蘋概股，GIS、美律權證亮~1/06
[公告] N5D6M6C 違反板規8 水桶10年~1/06
[公告] efun0242違反板規4-4 水桶一週~1/06
[請益] 近幾年像台積電一樣強勢的股票~1/06
[請益] 銀行保險業的動作，可判斷匯率走勢嗎？~1/06
[新聞] 友訊(2332)  利多帶動股價漲停~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 康控營收~1/06
[公告] bsexpb060137 違反板規4-4 水桶一週~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[請益] 愛榭克是不是台灣最強經濟分析師？~1/06
[新聞] 墾丁飯店歇業全怪蔡英文？財經專家曝倒~1/06
[心得] 百條心法系列：第一條~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 《天堂M》故障爆退款潮　課長們刷退破百~1/06
[請益] 財報狗網站~1/06
[請益] type-C在台灣真的有肉可以吃嗎?~1/06
[標的] 2395研華 多~1/06
[新聞] 《財經觀測站》跟著大股東進出準沒錯~1/06
[新聞] 進口車強勢進逼　裕隆執行長嚴凱泰：我~1/06
[標的] 6462 神盾~1/06
[新聞] 16檔大戶搶買 下周領攻11,000點~1/06
[新聞] 宏達電12月營收僅40億元，創近13年次低~1/06
[新聞] 國產車大降價 破格搶客 ~1/05
[新聞] 華孚2017Q4合併營收破9億 連6季刷新紀~1/05
Re: [新聞] 盤子笑了！花269萬終獲《天堂M》紫變~1/05
[心得] 現在只要喊漲價就可以飆！？~1/05
Re: [標的] 3025星通~1/05
Re: [心得] 現在只要喊漲價就可以飆！？~1/05
大盤解讀~1/0

Re: [請益] 台積電上300買認購權證是不是可以大賺!?~1/06
[請益] 還有人在用權證想發財嗎~1/06
Re: [請益] 期貨程式交易 有人接觸嗎~1/06
[新聞] 陽明、長榮海迎接小旺季~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[標的] 3081聯亞~1/06
[新聞]登不進《天堂M》不爽了　課長爆氣刷退58萬~1/06
Re: [新聞]逢低布局蘋概股，GIS、美律權證亮~1/06
[公告] N5D6M6C 違反板規8 水桶10年~1/06
[公告] efun0242違反板規4-4 水桶一週~1/06
[請益] 近幾年像台積電一樣強勢的股票~1/06
[請益] 銀行保險業的動作，可判斷匯率走勢嗎？~1/06
[新聞] 友訊(2332)  利多帶動股價漲停~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 康控營收~1/06
[公告] bsexpb060137 違反板規4-4 水桶一週~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[請益] 愛榭克是不是台灣最強經濟分析師？~1/06
[新聞] 墾丁飯店歇業全怪蔡英文？財經專家曝倒~1/06
[心得] 百條心法系列：第一條~1/06
Re: [心得] 本魯自分析師的告白：投資股票該做的4件事~1/06
[新聞] 《天堂M》故障爆退款潮　課長們刷退破百~1/06
[請益] 財報狗網站~1/06
[請益] type-C在台灣真的有肉可以吃嗎?~1/06
[標的] 2395研華 多~1/06
[新聞] 《財經觀測站》跟著大股東進出準沒錯~1/06
[新聞] 進口車強勢進逼　裕隆執行長嚴凱泰：我~1/06
[標的] 6462 神盾~1/06
[新聞] 16檔大戶搶買 下周領攻11,000點~1/06
[新聞] 宏達電12月營收僅40億元，創近13年次低~1/06
[新聞] 國產車大降價 破格搶客 ~1/05
[新聞] 華孚2017Q4合併營收破9億 連6季刷新紀~1/05
Re: [新聞] 盤子笑了！花269萬終獲《天堂M》紫變~1/05
[心得] 現在只要喊漲價就可以飆！？~1/05
Re: [標的] 3025星通~1/05
Re: [心得] 現在只要喊漲價就可以飆！？~1/05
大盤解讀~1/0

...

[標的] 2316 楠梓電~12/21
[閒聊] 2017/12/21 盤中閒聊~12/21
Re: [標的] 1323 永裕~12/21
Re: [標的] 本盟~12/21
Re: [標的] 1526 日馳(多)~12/21
[新聞] 金洲雙利空，今年EPS恐腰斬~12/21
[新聞]存股族借券一魚二吃 借券餘額市值衝5500億~12/20
[新聞] 美國參議院投票通過稅改議案！~12/20
Re: [標的] 2367 燿華~12/20
[其他] 106年12月20日三大法人買賣金額統計表~12/20
[新聞] 《金融股》玉山金攜手馬偕，緬甸義診~12/20
[新聞] 不甩蘋果遭降評 鴻海蘋概報復性反彈 重新~12/20
[新聞] 溢價逾倍！中美冠科今亮燈漲停60萬張瘋搶~12/20
[請益] 油價vs臺灣航空公司股價~12/20
Re: [標的]6257矽格（多）新聞消息面判斷~12/20
[新聞] 凱基投顧：3逆風恐觸發台股明年Q2轉空~12/20
[其他] 中國工業機器人2017年產量破12萬台、占~12/20
[請益] 中美冠科空單慘戶~12/20
[標的] 上證2X 00633L 多~12/20
[請益] 隔日沖做法~12/20
[請益] 如何看待目前的雞蛋水餃股？~12/20
[新聞]死道友不死貧道：還沒交屋就被套牢的預售~12/20
Re: [請益] 如何看待目前的雞蛋水餃股？~12/20
[標的] 2347 聯強~12/20
[標的] 8112 至上~12/20
[新聞] 嚇死人！郭董美國新廠是「吃電大怪獸」 ~12/20
[新聞] 日廠JSR擬每股75元併購冠科-KY 溢價104%~12/20
[閒聊] 收聽廣播電臺~12/20
[標的] 台積電 060302 多~12/20
[標的] 1463強盛怎麼一直跌~12/20
[新聞] 勞動基金 將加碼台股800億~12/20
[閒聊] 這輩子如果再也見不到四位數的台股？~12/20
[新聞] OPPO採購去台灣化 不利至上~12/20
Re: [新聞] 東哥掛牌上市 衝高湧賣壓~12/20
[新聞] 個股：接單太強勁，法人圈上修京鼎(3413)~12/20
[新聞] 網家多角化 擬設實體超市~12/20
Re: [標的] 本盟~12/20
[標的] 4129 聯合~12/20
[新聞] 《天堂Ｍ》伺

[標的] 8489 三貝德~12/22
Re: [標的] 本盟~12/22
[新聞] 中國A50正2ETF 規模破百億~12/22
[標的] 2367 燿華~12/22
[請益] 看到不錯的股票想放空 可是~12/22
[請益] 標借與空頭的關聯~12/22
Re: [請益] 為什麼有些事情跟觀念很難廣為人知~12/22
[標的] 6452康友-KY~12/22
Re: [請益] 為什麼有些事情跟觀念很難廣為人知~12/22
[標的] 2472 立隆電 多多~12/22
[新聞] 應景活動連發　橘子遊戲推聖誕小關卡~12/21
[新聞] 車用及Mini LED閃，億光、晶電後市旺~12/21
[新聞] 讓舊iPhone變慢？蘋果終於承認且給了一~12/21
Re: [請益] 有人試過用物理的相變化來解釋股市??~12/21
[新聞] 禾伸堂(3026)後市展望佳，股價再創波段新~12/21
[閒聊] 2017/12/21 盤後閒聊~12/21
[其他] 從印度調高進口關稅，將影響廠商全球布~12/21
Re: [請益] 有關股利與存股族問題~12/21
[新聞] DRAM市況熱 威剛全年拚賺1股本~12/21
[標的] 晶電(多)~12/21
[標的] 3558~12/21
[新聞] 宜進售杭州廠預計處分利益5.12億元 貢~12/21
[標的] 2534 宏盛~12/21
[其他] 106年12月21日三大法人買賣金額統計表~12/21
[請益] 證交所網站有限制每分鐘存取資料次數嗎?~12/21
[請益] 被動元件真的這麼厲害？~12/21
Re: [請益] 有人試過用物理的相變化來解釋股市??~12/21
Re: [請益] 中美冠科空單慘戶~12/21
Re: [請益] 有人試過用物理的相變化來解釋股市??~12/21
Re: [請益] 中美冠科空單慘戶~12/21
[請益] 有關股利與存股族問題~12/21
Re: [新聞] 美國參議院投票通過稅改議案！~12/21
[標的] 3299帛漢 多~12/21
[標的] 2316 楠梓電~12/21
[閒聊] 2017/12/21 盤中閒聊~12/21
Re: [標的] 1323 永裕~12/21
Re: [標的] 本盟~12/21
Re: [標的] 1526 日馳(多)~12/21
[新聞] 金洲雙利空，今年E

[新聞] 美律再申讓康控股票2000張 將獲利2.45億~12/12
Re: [請益] 大家對蔡明忠說物價漲薪水就會漲的看法？~12/12
[標的] 2809 京城銀 + 銀行績效分析~12/12
[新聞] 益安 106年11月營收594萬、年減65.07%~12/12
Re: 想開一個商學院的公司 請各位給意見 謝謝~12/12
[新聞] Google Home Max開賣 鴻海等供應鏈受惠~12/12
[新聞] 壽險今年敲進台股逾6200億 史上新高~12/12
[標的] 2367 燿華~12/12
[其他] 106年12月12日信用交易統計~12/12
[新聞] 橘子《天堂M》正式開服 登入數突破百萬人~12/12
[新聞] 台積電遭降評 張忠謀打臉外資~12/12
[新聞]南山人壽 每股純益2元~12/12
[新聞] 《金融股》元大「點金靈」大當機，IT緊-~12/12
[請益] 買大牛股但是每天都緊張到看盤30-50次~12/12
[新聞] 《熱門族群》製鞋三雄11月齊高歌~12/12
Re: [新聞] 藍委爆：3家太陽能廠虧損90億 國發基金竟~12/12
Re: [標的] 橘子(炸裂空)~12/12
Re: [標的] 康控 多~12/12
Re: [標的] 4180安成藥（出清10%獲利）~12/12
[標的] 3508 位速 空~12/12
想開一個商學院的公司 請各位給意見 謝謝~12/12
[新聞] 越南經濟起飛，LED 照明市場至 2020 年~12/12
[閒聊] 2017/12/12 盤後閒聊~12/12
[心得] 臺指期外資未平倉淨口數vs.臺股現貨~12/12
[其他] 106年12月12日三大法人買賣金額統計表~12/12
[心得] 低檔買進,長期持有,終於有點成績了~~12/12
Re: [請益] 全職交易出現的問題~12/12
[新聞] 鴻海拚耶誕旺季 iPhone X出貨神速~12/12
Re: [心得] 臺指期外資未平倉淨口數vs.臺股現貨~12/12
[其他] 橘子-屋頂版~12/12
Re: [新聞] 浩鼎中裕漲停領軍 生技股跌深反彈~12/12
[請益] 每天盤後功課~12/12
Re: [請益] 全職交易出現的問題~12/12
Re: [新聞] 藍委爆：3家太陽能廠虧損90億 國發基金竟~12/12
[請益] 台銀證券

[標的] 6133金橋 多短波段單~12/22
[新聞] 宏達電獲外資挺 股價谷底翻揚~12/22
Re: [標的] 8933愛地雅（漲停嘍！！）~12/22
Re: [請益] 中美冠科空單慘戶~12/22
[閒聊] 2017/12/22 盤後閒聊~12/22
[其他] 106年12月22日三大法人買賣金額統計表~12/22
[新聞] 台股盤後－電子三雄攜手曬冬陽 金融拉尾~12/22
Re: [請益] 為什麼有些事情跟觀念很難廣為人知~12/22
Fw: [新聞] 彭淮南護匯率政策 左批張忠謀右打蔡明忠~12/22
Re: [請益] 有人試過用物理的相變化來解釋股市??~12/22
Re: [新聞] 彭淮南護匯率政策 左批張忠謀右打蔡明忠~12/22
[新聞] 大同擬現金增資 折價發行幅度大 ~12/22
[請益] 股價反映的未來極限有多遠？~12/22
Re: [新聞] 大同擬現金增資 折價發行幅度大 ~12/22
Re: [標的] 2353宏碁~12/22
[請益] 當沖算是賭博嗎？~12/22
[新聞] 陳俊聖領軍 宏碁毛利率13年新高~12/22
Re: [請益] 為什麼有些事情跟觀念很難廣為人知~12/22
[標的] 6180 橘子~12/22
[新聞]《天堂》國際伺服器版本進行「支配者之塔 ~12/22
[其他] 如何分辨投資領域的江湖術士？~12/22
[新聞] 台船決減資再增資，明年拚國艦國造和風電~12/22
[閒聊] 2017/12/22 盤中閒聊~12/22
[新聞] 拍板了！2035年禁售燃油機車、2040年禁~12/22
Re: [新聞] 經長：電動機車產業鏈缺口　目前只剩電池~12/22
Re: [標的] 本盟~12/22
[標的] 8933愛地雅（個人進場13.05元）~12/22
[新聞] 台積再奪三星訂單！傳明年吃高通驍龍855~12/22
[標的] 1524 耿鼎~12/22
[標的] 8489 三貝德~12/22
Re: [標的] 本盟~12/22
[新聞] 中國A50正2ETF 規模破百億~12/22
[標的] 2367 燿華~12/22
[請益] 看到不錯的股票想放空 可是~12/22
[請益] 標借與空頭的關聯~12/22
Re: [請益] 為什麼有些事情跟觀念很難廣為人知~12/22
[標的] 6452康友-KY~12

[新聞] 陸行之 喊買台積電~12/08
[新聞] 台股恐跌破萬點 顧立雄：烏鴉嘴~12/07
[其他] 11月營收~12/07
[新聞] 《業績-金融》玉山金前11月EPS 1.41元~12/07
[標的] 6121 新普~12/07
Re: [新聞] 北極星藥業-KY:本公司召開法人說明會~12/07
[標的] 元大金 2885~12/07
[標的]  3227原相~12/07
[請益] 有關成交量大小的疑問~12/07
[新聞] 前11月EPS，兆豐金1.85元、玉山金1.41元~12/07
[新聞] 三安將成為全球最大LED龍頭 它還要加碼~12/07
[標的] 4721從高點腰斬但仍是起漲點3倍，敢買嗎?~12/07
[新聞] 聯發科技106年11月份自結合併營收淨額~12/07
[心得] 請做可以承受虧損的交易~12/07
Re: [新聞] 聯發科技106年11月份自結合併營收淨額~12/07
Re: [新聞] 三安將成為全球最大LED龍頭 它還要加碼~12/07
[其他] 106年12月07日信用交易統計~12/07
Re: [標的] 3498 陽程(空)~12/07
[新聞] 福特汽車董事長：中國將引領電動汽車的~12/07
Re: [請益] 股版大家真的有賠很多嗎？~12/07
Re: [請益] 晶電2448是不是差不多了~12/07
[新聞] 中石化受惠業績題材 股價爆量大漲 收復5~12/07
Re: [請益] 我聽了張忠謀說台積電10年不擔心才買股票~12/07
[新聞]先進光大立光侵權官司判決 先進光:將公告~12/07
Re: [心得] 致 跟我一樣省錢投資的年輕人~12/07
Re: [標的] TPK-KY(3673)  空~12/07
[標的] 神盾 「多」~12/07
[標的]2377微星~12/07
[標的] 6456空~12/07
[其他] 2017年第三季加密貨幣與區塊鏈更火熱~12/07
Re: [標的] 4939 亞電 (多)~12/07
[請益] 發言人的回覆和公司值不值得投資有關係嗎~12/07
[新聞] 旺宏11月營收~12/07
[標的] 光燿科(3428) 空~12/07
[其他] 106年12月07日三大法人買賣金額統計表~12/07
[請益] 營收報告後股價~12/07
[其他] VR的最後一搏？還是走到了盡

In [249]:
articles=DataFrame(articles)

In [250]:
Title = articles[0].str.split('~',expand=True).rename(columns={0:'articles',1:'date'})

In [251]:
Title

,articles,date,2
0,Re: [請益] 台積電上300買認購權證是不是可以大賺!?,1/06,None
1,[請益] 還有人在用權證想發財嗎,1/06,None
2,Re: [請益] 期貨程式交易 有人接觸嗎,1/06,None
3,[新聞] 陽明、長榮海迎接小旺季,1/06,None
4,Re: [心得] 本魯自分析師的告白：投資股票該做的4件事,1/06,None
5,[標的] 3081聯亞,1/06,None
6,[新聞]登不進《天堂M》不爽了 課長爆氣刷退58萬,1/06,None
7,Re: [新聞]逢低布局蘋概股，GIS、美律權證亮,1/06,None
8,[公告] N5D6M6C 違反板規8 水桶10年,1/06,None
9,[公告] efun0242違反板規4-4 水桶一週,1/06,None


In [256]:
def get_keywords(content):
    keywords=jieba.analyse.extract_tags(str(content),topK=15)
    print ("/".join(keywords))

In [259]:
group=Title.groupby('date')['articles'].apply(get_keywords)

959/低檔/買進/終於有點/成績/Name/dtype/object/心得/持有
新聞/請益/追高殺/低長/期能/Re/202/203/陸徵環/保稅/負擔/變重/204/嚴凱泰/無所適
Re/新聞/請益/02/股匯/雙漲/台幣/開盤/大升/9.3/01/2017/閒聊/心得/大手筆
新聞/Re/請益/03/閒聊/2018/01/榮海/日電貿/11/資金/股利/好奇/玩死/其他
新聞/Re/請益/04/01/2018/怎麼/穩懋/閒聊/3D/107/市場/獲利/3105/請問
新聞/Re/請益/05/心得/01/現在/漲價/大盤/閒聊/2018/國產車/盤子/269/萬終
新聞/請益/Re/告白/本魯自/師的/投資/心得/違反板規/水桶/分析/台積/權證/課長/10
Re/新聞/請益/心得/賠錢/心痛/11/12/06/怎麼/台股/高點/直銷/大盤/金管會
Re/新聞/請益/心得/11/一樣/投資/輕人/營收/106/鴻海/業績/EPS/1.41/KY
新聞/Re/請益/11/EPS/群創/營收/比特/106/12/08/幣破/關卡/5000/心得
新聞/11/Re/營收/請益/比特/台股/晶電/億元/10/家金控/賺贏/鴻海/攻頂/696
Re/請益/新聞/12/心得/全職/問題/橘子/11/出現/這盤/難解/營收/用心/海後市
新聞/Re/請益/11/12/橘子/登陸華爾街/比特/交易/作多/外資/精材/決在/兩岸/營收
新聞/Re/請益/12/11/106/外資/閒聊/橘子/看盤/汽車/盤後功/蔡明忠/說物/價漲
Re/請益/新聞/12/13/閒聊/其他/2809/銀行/績效/橘子/天堂/106/怎麼/當日
新聞/Re/橘子/12/14/6180/投資/2017/心得/蘋果/大陸/閒聊/天堂/國際/Finisar
新聞/Re/請益/天堂/12/15/百萬/工讀/閒聊/橘子/2405/浩鑫/拚亮/2017/台灣
新聞/Re/台灣/請益/DRAM/產業/當初/怎麼/韓國/台股/12/民眾估/萬點/六成/其他
Re/請益/綠角大/新聞/篇文章/台積/怎麼/有千萬現/股崩盤/iX/現雜音/A11/訂單/30/本週
新聞/Re/12/18/請益/心得/綠角/11/ETF/台灣/閒聊/45/東森/VR/怎麼
新聞/Re/12/19/閒聊/請益/股王/條款/2017/類股/

In [260]:
group

date
             None
1/01         None
1/02         None
1/03         None
1/04         None
1/05         None
1/06         None
12/06        None
12/07        None
12/08        None
12/09        None
12/10        None
12/11        None
12/12        None
12/13        None
12/14        None
12/15        None
12/16        None
12/17        None
12/18        None
12/19        None
12/20        None
12/21        None
12/22        None
12/23        None
12/24        None
12/25        None
12/26        None
12/27        None
12/28        None
12/29        None
12/30        None
12/31        None
12/6 短單分享    None
Name: articles, dtype: object